
<h2>Problem Statement</h2>

In this challenge, we invite Kagglers to help us identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted.

<br><br>
Submissions are scored on the <b>area under the ROC curve</b>. :

![area under the ROC curve](https://developers.google.com/machine-learning/crash-course/images/AUC.svg)

In [ ]:
#IMPORTING REQUIRED LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math

from lightgbm.sklearn import LGBMRegressor
from lightgbm.sklearn import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import roc_auc_score,accuracy_score

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

import gc
gc.enable()


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#All functions

#FUNCTION FOR PROVIDING FEATURE SUMMARY
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        df.at[col,'Sample_values']=list(df_fa[col].unique())
    display(df_fa.head())   
    return(df.fillna('-'))



In [ ]:
#DATASET VIEW
path='../input/'
data_files=list(os.listdir(path))
df_files=pd.DataFrame(data_files,columns=['File_Name'])
df_files['Size_in_MB']=df_files.File_Name.apply(lambda x:round(os.stat(path+x).st_size/(1024*1024),2))
df_files

In [ ]:
%%time
#READING ONLY TRAINING SET SAMPLE
print('reading train dataset...')
df_train=pd.read_csv(path+'train.csv',nrows=100000)
# print('reading test dataset...')
# df_test=pd.read_csv(path+'test.csv',nrows=100000)
# print('submission file')
# df_submission=pd.read_csv(path+'sample_submission.csv')

In [ ]:
#CREATING AND STORING FEATURE SUMMARY
fs_train=feature_summary(df_train)
with pd.option_context('display.max_rows', 999):
    display(fs_train)

In [ ]:
feature_list=list(fs_train.index)
feature_list.remove('MachineIdentifier')
feature_list.remove('HasDetections')
feature_list.remove('AvSigVersion')
feature_list.remove('OsBuildLab')
feature_list.remove('Census_OSVersion')

In [ ]:
%%time
fs_train['score']=0
model=LGBMClassifier(learning_rate=0.05,n_estimators=100,n_jobs=-1,reg_alpha=0.1,min_split_gain=.1,verbose=-1)
for feature in feature_list:
    df=pd.read_csv(path+'train.csv',usecols=['HasDetections',feature])
    X=df[feature]
    y=df['HasDetections']
    if fs_train.loc[feature,'Data_type']!='object':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(np.array(X_train).reshape(-1,1),y_train)
        score=roc_auc_score(y_test,(model.predict_proba(np.array(X_test).reshape(-1,1))[:,1]))
        
    else:
        X=pd.get_dummies(df[feature],prefix=feature).values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train)
        score=roc_auc_score(y_test,(model.predict_proba(X_test)[:,1]))
        
    fs_train.at[feature,'score']=score
    print('feature:',feature,'\tscore:\t',score,'\tDatatype:\t',fs_train.loc[feature,'Data_type'])
    gc.collect()
    
    

In [ ]:
#WRITING FS_TRAIN FILE 
fs_train.to_csv('feature_score.csv',index=False)

In [ ]:
# print(X.shape,y.shape,test.shape)

In [ ]:
# %%time
# #CREATING FINAL MODEL WITH STRATIFIED KFOLDS
# #FOLD COUNT 10
# #TRIED XGBClassifier, LGBMClassifier, CatBoostClassifier
# #BEST SCORE ACHIEVED BY CatBoostClassifier

# model=CatBoostClassifier(iterations=1000,
#                               learning_rate=0.05,
#                               depth=7,
#                               l2_leaf_reg=40,
#                               bootstrap_type='Bernoulli',
#                               subsample=0.7,
#                               scale_pos_weight=5,
#                               eval_metric='AUC',
#                               metric_period=50,
#                               od_type='Iter',
#                               od_wait=45,
#                               random_seed=17,
#                               allow_writing_files=False)

# # model= XGBClassifier(
# #  learning_rate =0.1,
# #  n_estimators=1000,
# #  max_depth=5,
# #  min_child_weight=1,
# #  gamma=0,
# #  subsample=0.8,
# #  colsample_bytree=0.8,
# #  objective= 'binary:logistic',
# #  nthread=8,
# #  scale_pos_weight=1,
# #  seed=27,
# #  n_jobs=-1)

# # model=LGBMClassifier(
# #             nthread=4,
# #             n_estimators=1000,
# #             learning_rate=0.02,
# #             num_leaves=34,
# #             colsample_bytree=0.9497036,
# #             subsample=0.8715623,
# #             max_depth=8,
# #             reg_alpha=0.041545473,
# #             reg_lambda=0.0735294,
# #             min_split_gain=0.0222415,
# #             min_child_weight=39.3259775,
# #             silent=-1,
# #             verbose=-1)

# #DATAFRAMES FOR STORING PREDICTIONS ON TRAIN DATA AS WELL AS TEST DATA
# #CAN BE USED FOR ENSEMBLE 
# df_preds=pd.DataFrame()
# df_preds_x=pd.DataFrame()
# k=1
# splits=10
# avg_score=0

# #CREATING STRATIFIED FOLDS
# skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=200)
# print('\nStarting KFold iterations...')
# for train_index,test_index in skf.split(X,y):
#     df_X=X.iloc[train_index,:]
#     df_y=y.iloc[train_index]
#     val_X=X.iloc[test_index,:]
#     val_y=y.iloc[test_index]

# #FITTING MODEL
#     model.fit(df_X,df_y)

# #PREDICTING ON VALIDATION DATA
#     col_name='cat_predsx_'+str(k)
#     preds_x=pd.Series(model.predict_proba(val_X)[:,1])
#     df_preds_x[col_name]=pd.Series(model.predict_proba(X)[:,1])

# #CALCULATING ACCURACY
#     acc=roc_auc_score(val_y,preds_x)
#     print('Iteration:',k,'  roc_auc_score:',acc)
#     if k==1:
#         score=acc
#         model1=model
#         preds=pd.Series(model.predict_proba(test)[:,1])
#         col_name='cat_preds_'+str(k)
#         df_preds[col_name]=preds
#     else:
#         preds1=pd.Series(model.predict_proba(test)[:,1])
#         preds=preds+preds1
#         col_name='cat_preds_'+str(k)
#         df_preds[col_name]=preds1
#         if score<acc:
#             score=acc
#             model1=model
#     avg_score=avg_score+acc        
#     k=k+1
# print('\n Best score:',score,' Avg Score:',avg_score/splits)
# #TAKING AVERAGE OF PREDICTIONS
# preds=preds/splits

In [ ]:
#PREPARING SUBMISSION
# df_submission['target']=preds
# df_submission

In [ ]:
#CREATING SUMBISSION FILE
# df_submission.to_csv('submission.csv',index=False)

In [ ]:
# svc = svm.SVC(kernel='linear', C=1,probability=True).fit(df_preds_x,y)

In [ ]:
# df_submission2=df_submission.copy()
# df_submission2['target']=svc.predict_proba(df_preds)[:,1]

# df_submission2

In [ ]:
# df_submission2.to_csv('submission2.csv',index=False)